# 7.2 Normal distribution naive Bayes classifier

In machine learning, naive Bayes classifiers are a family of simple "probabilistic classifiers" based on applying Bayes' theorem with strong (naïve) independence assumptions between the features:$\\[10pt]$

<img src="./images/bayes.png" width="700">

A **normal distribution naive Bayes classifier** supposes that input parameters follow the probability density function of a normal distribution:$\\[5pt]$


$$p(\mathbf{x}/C_i) = \large{\frac{1}{(2\pi)^{n/2}|\Sigma^i|^{1/2}}}e^{-(\mathbf{x}-\mu^i)^T\Sigma_i^{-1}\mathbf{x}-\mu^i)}\\[5pt]$$

In this notebook, will explain how Bayesian classifiers work (section 7.2.1) and, specifically, how to classify a feature vector supposing that follows a normal distribution (section 7.2.2).

*Naive Bayes classifier is a Bayesian classifier where independence between all variables is assumed. In this chapter we will work only with naive classifiers.*

## Problem context - Traffic sign recognition

In the previous notebook, *AliquindoiCars* contacted us for a TSR technique to be integrated into a self-driving car. They provided us with some segmented images of traffic signs that their autonomous cars captured during test drivings. These images are located in `images/circles/` containing circled signs, `images/triangles` containing signs having triangle shapes and `images/squares` containing signs having square shapes.$\\[3pt]$

<img src="./images/signs.jpg" width="400">$\\[3pt]$

Previously, we extracted a feature vector for each data image using Hu moments. Now, we will train a classifier using the feature vectors we saved in previous notebook. For classification, there are many methods we can apply, such as [kNN algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm), [random forest](https://en.wikipedia.org/wiki/Random_forest), ... In this notebook, we will learn and use a naive Bayes classifier.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 8.0)

images_path = './images/'
import sys
sys.path.append("..")
from utils.PlotEllipse import PlotEllipse

### 7.2.1 Bayesian classifier

Having a set of data variables $\mathbf{x} = [x_1, x_2, x_3, \ldots, x_n]^T$ and a set of possible classes $\mathbf{C} = [C_1, C_2, C_3, \ldots, C_n]$. The bayesian classifier assigns $\mathbf{x}$ to the class $C_i$ that has the highest posterior probability $P(C_i/x)$ (the more probable, the less probability of making a mistake). This is called a **MAP** prediction:$\\[3pt]$

<img src="./images/MAP.png" width="400">$\\[3pt]$

In this example, $\mathbf{x}$ have only one variable. Deppending on the value of the variable, $\mathbf{x}$ (feature vector of some object) is assigned to a class or not. Sometimes, it is convenient to have a *rejection region*, where no decision is made (e.g. $P(C_1/x) = P(C_2/x) = 0.5$).$\\[3pt]$

<img src="./images/rejectregion.png" width="300">$\\[3pt]$

**Discriminant function**

We can obtain a discriminant function $d_i(x)$ for each class $C_i$, such that $d_i(x) \gt d_j(x)$ whenever $P(C_i/x) \gt P(C_j/x)$ from the Bayes theorem:$\\[10pt]$

$$d_k(x) = P(C_k/\mathbf{x}) = \frac{p(\mathbf{x}/C_k)P(C_k)}{P(\mathbf{x})}\\[5pt]$$

This is, a way to compute the posterior probability $P(C_k/\mathbf{x})$ of a feature vector $\mathbf{x}$ for each class. Anyways, we can simplify this function:$\\[10pt]$

$$
\begin{eqnarray}
&d_k(x) &= P(C_k/\mathbf{x}) = \frac{p(\mathbf{x}/C_k)P(C_k)}{P(\mathbf{x})}\\
\xrightarrow{P(\mathbf{x})\text{ is a constant value }\forall k} \hspace{0.2cm}&d_k(x) &= p(\mathbf{x}/C_k)P(C_k)\\[7pt]
\xrightarrow{\text{ ln doesn't alter maximum }} \hspace{0.2cm} &d_k(x) &= lnp(\mathbf{x}/C_k) + lnP(C_k)\\[7pt]
\xrightarrow{\text{If } P(C_k) = P(C_j)\; \forall \ j,k }\hspace{0.2cm} &d_k(x) &= lnp(\mathbf{x}/C_k)
\end{eqnarray}
\\[8pt]$$

This is called **maximum log-likelihood estimation**.

### 7.2.2 Naive Bayes classifier for normal distribution

We can suppose that features follow a normal distribution when applying a Bayesian classifier.$\\[5pt]$

$$p(\mathbf{x}/C_i) = \large{\frac{1}{(2\pi)^{n/2}|\Sigma^i|^{1/2}}}e^{-(\mathbf{x}-\mu^i)^T\Sigma_i^{-1}\mathbf{x}-\mu^i)}\\[5pt]$$

In that way, the discriminant function:

$$
\begin{eqnarray}
d_k(\mathbf{x}) &=& ln\ P(C_k) + ln\ p(\mathbf{x}/C_k) = ln\ P(C_k) + ln\ \frac{1}{(2\pi)^{n/2}|\Sigma^i|^{1/2}}e^{\color{red}{-(\mathbf{x}-\mu^i)^T\Sigma_i^{-1}\mathbf{x}-\mu^i)}}\\[7pt]
    &=& ln\ P(C_k) - ln\ 2\pi^{n/2} - ln\ |\Sigma^k|^{1/2} - \frac{1}{2} \color{red}{D_k^2(x)}\\[7pt]
    &=& ln\ P(C_k) - \frac{1}{2}\left[n ln\ 2\pi +  ln\ |\Sigma^k| + D_k^2(x)\right] \xrightarrow{\text{cte can be removed}} ln\ P(C_k) - \frac{1}{2}\left[ln\ |\Sigma^k| + D_k^2(x)\right]\\[7pt]
\end{eqnarray}
\\[8pt]$$

We can see that the **discriminant function is quadratic**:

$$d_k(\mathbf{x}) = ln\ P(C_k) - \underbrace{ \frac{1}{2} ln\ |\Sigma^k| - \left[(\mathbf{x-\mu})^T (\Sigma^k)^{-1} (\mathbf{x-\mu}) \right]}_{\text{Independent term}} + \mathbf{x}^T (\underbrace{\Sigma^k)^{-1}\mathbf{\mu}^k}_{\text{linear term}} - \frac{1}{2} \mathbf{x}^T \underbrace{(\Sigma^k)^{-1}}_{\text{Quadratic term}}\mathbf{x}$$

Visually, **division boundaries are parabolas**:

<img src="./images/visual_normal_bayes.png" width="600">$\\[3pt]$


### **<span style="color:green"><b><i>ASSIGNMENT 1a</i></b></span>**

Now, we are going to implement this naive bayes classifier for normal distributions using the Hu moments computed in the previous exercise.

**Training the classifier**

The first step is computing the weight matrix of the discriminant function. In this case, it depends on the **means** ($\bf{\mu}$) and **covariance matrix** ($\Sigma$) of the training data.

In the previous exercise we proved that out problem can be solved using only the first and second Hu moments. **Your first task** is to load (using [np.load](https://numpy.org/doc/stable/reference/generated/numpy.load.html?highlight=load#numpy.load)) the firsts two Hu moments of each class' data, which was computed in previous notebook. Then, **compute the means** (or centroid) and **covariance matrix** for each class.

*We can compute the covariance matrix of a set of points using [np.cov](https://numpy.org/doc/stable/reference/generated/numpy.cov.html).*

In [ ]:
# Load first 2 Hu moments of each class 


# Compute covariance matrices


# Compute means



### **<span style="color:green"><b><i>ASSIGNMENT 1b</i></b></span>**

Your **next task** is to develop a method `discriminant_function`, that computes $d_k(x)$. The inputs have to be:

- `features`: feature vector of dimension n
- `mu`: mean vector of the class k of which is being computed the probability
- `cov`: covariance matrix with shape (n,n) of the class k
- `prior`: prior probability of class k

The method should evaluate (then return) the discriminant function.

In [ ]:
def discriminant_function(features, mu, cov, prior):
    """ Evaluates the discriminant function d(x)
    
        Args:
            features: feature vector of dimension n
            mu: mean vector of the class of which is being computed the probability
            cov: covariance matrix with shape (n,n) of the class
            prior: prior probability of class k

        Returns:
            dx: result of discriminant function
    """

    

### **<span style="color:green"><b><i>ASSIGNMENT 1c</i></b></span>**

**Testing the classifier**

For testing if the classifier works, we are going to classify some new images and check if the classifier is correct or not. *Note that the discriminant function is the logarithm of a probability, not a probability itself (values can be positive and negatives, but the result of max function is the same)*.

For this, evaluate the images `train_circle.png`,`train_square.png` and `train_triangle.png` using the discriminant function with each class and choosing the maximum one.

**What to do?** Compute the Hu moments of each training image and apply the classifier to detect their class (using the previous method). Try to construct a method that classify any image in order to save code.

*We assume that there is no prior information of any class $P(C_i) = P(C_j)\; \forall i,j$. This is because a priori, we see the same number of circle, square and triangle shaped road signs when driving.*


In [ ]:
def image_moments(region):
    """ Compute moments of the external contour in a binary image.   
    
        Args:
            region: Binary image
                    
        Returns: 
            moments: dictionary containing all moments of the region
    """   
    
    # Get external contour
    
    
    # Compute moments
    
    

In [ ]:
def classify_image(sign_image):
    """ Classify a traffic sign image by its shape using a bayesian classifier   
    
        Args:
            sign_image: Binarized image
    """   
    
    # Compute Hu moments
    
    
    # Classify circle test image
    
    
    # Search the maximum
    

In [ ]:
# Read images
test_circle = cv2.imread(images_path + "test_circle.png", 0)
test_triangle = cv2.imread(images_path + "test_triangle.png", 0)
test_square = cv2.imread(images_path + "test_square.png", 0)

# Classify them
print("Circle: ")
classify_image(test_circle)
print("Triangle: ")
classify_image(test_triangle)
print("Square: ")
classify_image(test_square)

Finally, we can see how this classifier divide the feature space showing the covariance ellipses. **You have to modify the names of the variables** on this code to make it works, showing the covariance ellipces of each class.

In [ ]:
# Create figure
fig, ax = plt.subplots()
plt.axis([0.14, 0.2, -0.01, 0.059])

# Plot hu moments
plt.plot(train_triangles[0,:],train_triangles[1,:],'go')
plt.plot(train_circles[0,:],train_circles[1,:],'ro')
plt.plot(train_squares[0,:],train_squares[1,:],'bo')

# Plot ellipses representing covariance matrices
PlotEllipse(fig, ax, np.vstack(mean_triangles), cov_triangles, 5, color='black')
PlotEllipse(fig, ax, np.vstack(mean_squares), cov_squares, 5, color='black')
PlotEllipse(fig, ax, np.vstack(mean_circles), cov_circles, 5, color='black')

fig.canvas.draw()

### **<span style="color:green"><b><i>ASSIGNMENT 2</i></b></span>**

**Simplification**

We can simplify our classifier using Euclidean distance instead of Mahalanobis one. This can be achieved using isotropic covariance matrices:

$$\Sigma^k = \Sigma = \sigma^2 \cdot I = \sigma^2 \begin{bmatrix}1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1\end{bmatrix}$$

In this way, decision boundaries are lines, and covariances are spherical. This is called a **natural classifier**:$\\[5pt]$

<img src="./images/natural_classifier.png" width="500">$\\[3pt]$

The discriminant function can be simplified, and the cuadratic term disappear:$\\[6pt]$

$$d_k(x) = -(\mathbf{x-\mu^k})^T(\mathbf{x-\mu^k}) = -||\mathbf{x-\mu^k}||^2\\[6pt]$$

**What to do?** Repeat the previous exercise using isotropic covariance matrices.

In [ ]:
def discriminant_function_isotropic(features, mu):
    """ Evaluates the discriminant function of a naive Bayes clasifier using isotropic covariances
    
        Args:
            features: feature vector of dimension n
            mu: mean vector of the class of which is being computed the probability

        Returns:
            dx: result of discriminant function
    """
    

In [ ]:
def classify_image_isotropic(sign_image):
    """ Classify a traffic sign image by its shape using a bayesian classifier   
    
        Args:
            sign_image: Binarized image
    """   
    
    # Compute Hu moments
    
    
    # Classify circle test image
    
    
    # Search the maximum
    

In [ ]:
# Read images
test_circle = cv2.imread(images_path + "test_circle.png", 0)
test_triangle = cv2.imread(images_path + "test_triangle.png", 0)
test_square = cv2.imread(images_path + "test_square.png", 0)

# Classify them
print("Circle: ")
classify_image_isotropic(test_circle)
print("Triangle: ")
classify_image_isotropic(test_triangle)
print("Square: ")
classify_image_isotropic(test_square)

In [ ]:
# Create figure
fig, ax = plt.subplots()
plt.axis([0.14, 0.2, -0.01, 0.059])

# Plot hu moments
plt.plot(train_triangles[0,:],train_triangles[1,:],'go')
plt.plot(train_circles[0,:],train_circles[1,:],'ro')
plt.plot(train_squares[0,:],train_squares[1,:],'bo')

# Plot ellipses representing covariance matrices
PlotEllipse(fig, ax, np.vstack(mean_triangles), np.eye(2), 0.003, color='black')
PlotEllipse(fig, ax, np.vstack(mean_squares), np.eye(2), 0.003, color='black')
PlotEllipse(fig, ax, np.vstack(mean_circles), np.eye(2), 0.003, color='black')

fig.canvas.draw()

### <font color="blue"><b><i>Thinking about it (1)</i></b></font>

Now, **answer the following questions**:

- What are the pros and cons of using isotropic covariances?
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- In what type of problems could isotropic matrices be used?
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

## Conclusion

Awesome! You know now how to make a classifier of **previously segmented objects**. Note that for more complex shapes, you can use the **7 Hu moments instead of the two that we used** for visualization purposes and simplicity of the problem.

In this notebook you have learn to:

- construct a naive Bayes classifier and apply it to a real problem where variables follow a normal distribuion
- construct a simplified classifier where isotropic covariances are assumed
- improve a classifier (if needed) using rejection regions